In [2]:
import math


countries = ["Germany", "France", "Holand", "Belgium", "Luxemburg", "England", "Austria", "Switzerland", "Sweden", "Norway"]

def getMeanValue(lst: list) -> float:
    return float(sum(lst) / len(lst))

def getStandardDeviation(lst: list):
    a = 0

    for i in range(len(lst)):
        n = (lst[i] - getMeanValue(lst)) ** 2
        a = a + n

    return math.sqrt(a / (len(lst) - 1))

def printFormattedMatrix(matrix: list):
    for row in matrix:
        print(row)
    print(row)

def transposeMatrix(matrix: list) -> list:
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))] 

def normalizeMatrix(matrix: list) -> list:
    tmpMatrix = transposeMatrix(matrix)    
    finalMatrix = []  

    for i in range(len(tmpMatrix)):
        tmpColumn = []
        for j in range(len(tmpMatrix[i])):
            tmpColumn+=[((tmpMatrix[i][j] -min(tmpMatrix[i])) /( max(tmpMatrix[i])-min(tmpMatrix[i])))]
    
        finalMatrix.append(tmpColumn)
    
    return transposeMatrix(finalMatrix)

def calculateManhattanDistance(x: list, y: list) -> float:
    distance = 0

    for i in range(len(x)):
        distance += abs(x[i] - y[i])

    return distance

def getDistanceMatrix(matrix: list) -> list:
    tmpMatrix = []
    
    for i in range(len(matrix)):
        tmpRow = []
        for j in range(len(matrix)):
            tmpRow.append(calculateManhattanDistance(matrix[i], matrix[j]))

        tmpMatrix.append(tmpRow)

    return tmpMatrix

def findMinNonzeroElement(lst: list) -> float:
    tmpLst = []
    for element in lst:
        if element != 0:
            tmpLst.append(element)

    return min(tmpLst)

def findMinNonzeroElementIndex(lst: list) -> int:
    minElement = findMinNonzeroElement(lst)
    return lst.index(minElement)

def findClosesClusters(dist_matrix: list) -> tuple:
    row_min_vals = []
    
    for i in range(len(dist_matrix)):
        
        # finding row with minimal value
        
        row_min = dist_matrix[i][findMinNonzeroElementIndex(dist_matrix[i])]
        
        row_min_vals.append(row_min)
        
    row_min_ind = findMinNonzeroElementIndex(row_min_vals) # Cluster A
    
    
    # finding column-wise index of minimal value in row_min
    
    row_min = dist_matrix[row_min_ind] 
    
    col_min_ind = findMinNonzeroElementIndex(row_min) # Cluster B
    
    branch_length = dist_matrix[row_min_ind][col_min_ind] / 2 # Distance
    
    return row_min_ind, col_min_ind, branch_length

def truncateMatrix(matrix: list, truncIdx: int, on='rows') -> list:
    truncMatrix = []

    if on == 'rows':
        for idx, value in enumerate(matrix):
            if idx != truncIdx:
                truncMatrix.append(value)
    elif on == 'cols':
        matrix = transposeMatrix(matrix)
        for idx, value in enumerate(matrix):
            if idx != truncIdx:
                truncMatrix.append(value)   

        truncMatrix = transposeMatrix(truncMatrix)

    return truncMatrix

def mergeClusters(distanceMatrix: list, clusterA: list, clusterB: list) -> list:
    
    new_matrix = []
    
    for i in range(len(distanceMatrix)):
        
        new_row = []
        
        for j in range(len(distanceMatrix[0])):
            
            if i not in (clusterA, clusterB) and j not in (clusterA, clusterB):
                new_row.append(distanceMatrix[i][j])
                
            else:
                
                new_row.append(None)
                
        new_matrix.append(new_row)
        
    cls_to_merge_ID = min(clusterA, clusterB)
    cls_to_remove_ID = max(clusterA, clusterB)
        
    # deleting row for a cluster that we're dropping
    new_matrix.pop(cls_to_remove_ID)
    # since we're deleting row with index cls_to_remove_ID, we need to keep that in mind when calling distanceMatrix later (index += 1)
    
    # deleting column for a cluster that we're dropping
    new_matrix = [row[0:cls_to_remove_ID] + row[cls_to_remove_ID+1:] for row in new_matrix]
    
    for i in range(len(new_matrix)):
        for j in range(len(new_matrix[0])):
            
            if i == j:
                new_matrix[i][j] = 0
            
            elif not new_matrix[i][j]:
                
                if i >= cls_to_remove_ID:
                    
                    min_dist = min(distanceMatrix[i+1][cls_to_merge_ID], distanceMatrix[i+1][cls_to_remove_ID])
                    
                    new_matrix[i][j] = min_dist
                    new_matrix[j][i] = min_dist
                    
                else:
                    
                    min_dist = min(distanceMatrix[i][cls_to_merge_ID], distanceMatrix[i][cls_to_remove_ID])
                    
                    new_matrix[i][j] = min_dist
                    new_matrix[j][i] = min_dist
    
    return new_matrix    

def getClustersTree(distanceMatrix: list):
    linkageMatrices = []
    mergedClusters = []
    branchLengths = []
    steps = 0

    while len(distanceMatrix) > 1:
        linkageMatrices.append(distanceMatrix)
        clusterA, clusterB, distance = findClosesClusters(distanceMatrix)
        distanceMatrix = mergeClusters(distanceMatrix, clusterA, clusterB)
        steps += 1
        branchLengths.append(distance)
        print("Merging next clusters: ")
        print(countries[clusterA])
        print(countries[clusterB])
        print()
        mergedClusters.append([clusterA, clusterB])

        countries[clusterA] = '[' + countries[clusterA] + ' ' + countries[clusterB] + ']'
        countries.remove(countries[clusterB])

    linkageMatrices.append(distanceMatrix)

if __name__ == '__main__':
    matrix = [[90, 49, 88, 19, 57, 51, 19, 21, 27, 21, 81, 75, 44, 71, 22, 91, 85, 74, 30, 26],
              [88, 42, 63, 4, 76, 53, 11, 23, 11, 5, 87, 84, 40, 45, 88, 94, 47, 36, 57, 3],
              [96, 62, 98, 32, 62, 67, 43, 7, 14, 14, 83, 89, 61, 81, 15, 31, 97, 13, 53, 15],
              [94, 38, 48, 11, 74, 37, 23, 9, 13, 12, 76, 76, 42, 57, 29, 84, 80, 83, 20, 5],
              [97, 61, 86, 28, 79, 73, 12, 7, 26, 23, 85, 94, 83, 20, 91, 94, 94, 84, 31, 24],
              [27, 86, 99, 22, 91, 55, 76, 17, 20, 24, 76, 68, 89, 91, 11, 95, 94, 57, 11, 28],
              [55, 31, 61, 15, 29, 33, 1, 5, 15, 11, 49, 42, 14, 41, 51, 51, 72, 28, 13, 11],
              [73, 72, 85, 25, 31, 69, 10, 17, 19, 15, 79, 70, 46, 61, 64, 82, 48, 61, 48, 30],
              [97, 13, 93, 31, 60, 43, 43, 39, 54, 45, 56, 78, 53, 75, 9, 68, 32, 48, 2, 93],
              [92, 17, 83, 13, 62, 51, 4, 17, 30, 15, 61, 72, 34, 51, 11, 63, 94, 28, 2, 62]]

    printFormattedMatrix(matrix)
    print()

    matrix = normalizeMatrix(matrix)
    printFormattedMatrix(matrix)
    print()

    matrix = getDistanceMatrix(matrix)
    printFormattedMatrix(matrix)
    print()

    getClustersTree(matrix)

[90, 49, 88, 19, 57, 51, 19, 21, 27, 21, 81, 75, 44, 71, 22, 91, 85, 74, 30, 26]
[88, 42, 63, 4, 76, 53, 11, 23, 11, 5, 87, 84, 40, 45, 88, 94, 47, 36, 57, 3]
[96, 62, 98, 32, 62, 67, 43, 7, 14, 14, 83, 89, 61, 81, 15, 31, 97, 13, 53, 15]
[94, 38, 48, 11, 74, 37, 23, 9, 13, 12, 76, 76, 42, 57, 29, 84, 80, 83, 20, 5]
[97, 61, 86, 28, 79, 73, 12, 7, 26, 23, 85, 94, 83, 20, 91, 94, 94, 84, 31, 24]
[27, 86, 99, 22, 91, 55, 76, 17, 20, 24, 76, 68, 89, 91, 11, 95, 94, 57, 11, 28]
[55, 31, 61, 15, 29, 33, 1, 5, 15, 11, 49, 42, 14, 41, 51, 51, 72, 28, 13, 11]
[73, 72, 85, 25, 31, 69, 10, 17, 19, 15, 79, 70, 46, 61, 64, 82, 48, 61, 48, 30]
[97, 13, 93, 31, 60, 43, 43, 39, 54, 45, 56, 78, 53, 75, 9, 68, 32, 48, 2, 93]
[92, 17, 83, 13, 62, 51, 4, 17, 30, 15, 61, 72, 34, 51, 11, 63, 94, 28, 2, 62]
[92, 17, 83, 13, 62, 51, 4, 17, 30, 15, 61, 72, 34, 51, 11, 63, 94, 28, 2, 62]

[0.9, 0.4931506849315068, 0.7843137254901961, 0.5357142857142857, 0.45161290322580644, 0.45, 0.24, 0.47058823529411764, 0.3